In [3]:
import json
from pyspark.sql.functions import col, year, sum as Fsum

In [5]:
import json
import os
from pyspark.sql.functions import col, year, sum as Fsum
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# ==========================================
# 1. CONFIGURATION
# ==========================================
# Le dossier contenant vos partitions (date=2023-01-01, etc.)
# Spark va lire récursivement tout ce qu'il y a dedans.
input_path = "data/silver/silver_measures_union"

# Le fichier de sortie
output_json = "data/bronze/export_velo_par_annee.json"

print(f" Lecture des données depuis : {input_path}")

# ==========================================
# 2. LECTURE ET AGRÉGATION SPARK
# ==========================================
df = spark.read.parquet(input_path)

# On vérifie que les colonnes sont bien là
# Rappel : vous avez uniformisé le nom en 'point_id' et le comptage en 'flux'
df_stats = df.withColumn("annee", year(col("date"))) \
    .groupBy("annee", "point_id") \
    .agg(Fsum("flux").alias("total_velo")) \
    .orderBy("annee", "point_id")

# ==========================================
# 3. CONSTRUCTION DU JSON
# ==========================================
print(" Calcul des statistiques en cours...")
rows = df_stats.collect()

# Structure cible : { "2023": { "site_A": 1000, "site_B": 2000 }, "2024": ... }
json_output = {}

for row in rows:
    # On ignore les éventuelles lignes invalides
    if row["annee"] is None or row["point_id"] is None:
        continue
        
    annee_str = str(row["annee"])
    site_id = str(row["point_id"])
    total = int(row["total_velo"]) # On force en int Python standard pour le JSON
    
    # Si l'année n'existe pas encore dans le dictionnaire, on l'ajoute
    if annee_str not in json_output:
        json_output[annee_str] = {}
    
    # On ajoute le compteur pour cette année
    json_output[annee_str][site_id] = total

# ==========================================
# 4. ÉCRITURE DU FICHIER
# ==========================================
print(f" Écriture du fichier JSON...")

with open(output_json, "w", encoding="utf-8") as f:
    json.dump(json_output, f, indent=4)

print(f" SUCCÈS ! Fichier généré : {output_json}")

 Lecture des données depuis : data/silver/silver_measures_union
 Calcul des statistiques en cours...
 Écriture du fichier JSON...
 SUCCÈS ! Fichier généré : data/bronze/export_velo_par_annee.json
